In [ ]:
# File locations are hard coded. Please upload the dataset manually to the runtime before executing.

In [ ]:
# Import libraries
import os
import json
import glob
import shutil

In [ ]:
# Define directories
COCO_TEST_DIR = "Fabric Inspections.v1i.coco/test/"
COCO_TRAIN_DIR = "Fabric Inspections.v1i.coco/train/"
COCO_VAL_DIR = "Fabric Inspections.v1i.coco/valid/"

YOLO_TEST_LABEL_DIR = 'dataset/test/labels/'
YOLO_TRAIN_LABEL_DIR = 'dataset/train/labels/'
YOLO_VAL_LABEL_DIR = 'dataset/valid/labels/'

YOLO_TEST_IMAGE_DIR = 'dataset/test/images'
YOLO_TRAIN_IMAGE_DIR = 'dataset/train/images'
YOLO_VAL_IMAGE_DIR = 'dataset/valid/images'

In [ ]:
# Write to .txt where annotations "image_id" is equal to image "id"
def create_labels(coco_dir, yolo_dir):
    with open(coco_dir + "_annotations.coco.json", "r") as file:
        data = json.load(file)

    for num_img, i in enumerate(data['images']):
        for num_ano, j in enumerate(data['annotations']):
            if data['annotations'][num_ano]['image_id'] == data['images'][num_img]['id']:
                write_to_txt(num_img, num_ano, yolo_dir, data)

In [ ]:
# Get data from json and convert coco measurments to yolo using a formula
def write_to_txt(num_img, num_ano, yolo_dir, data):
    file = open(yolo_dir + data['images'][num_img]['file_name'] + '.txt', 'a+')

    category = data['annotations'][num_ano]['category_id'] - 1 # Yolo classes starts at 0 (coco starts from 1)
    bbox = data['annotations'][num_ano]['bbox']
    x_center = bbox[0]
    y_center = bbox[1]
    width = bbox[2]
    height = bbox[3]

    img_width = data['images'][num_img]['width']
    img_height = data['images'][num_img]['height']

    x_center = x_center / img_width
    y_center = y_center / img_height
    width = width / img_width
    height = height / img_height

    x_center = format(x_center, '.6f')
    y_center = format(y_center, '.6f')
    width = format(width, '.6f')
    height = format(height, '.6f')

    file.write(f"{category} {x_center} {y_center} {width} {height}\n")
    file.close()

In [ ]:
# Copy images to the yolo directory
def copy_images(source, destination):
    for jpgfile in glob.iglob(os.path.join(source, "*.jpg")):
        shutil.copy(jpgfile, destination)

In [ ]:
# Create yolo directories
yolo_dirs = [YOLO_TEST_IMAGE_DIR, YOLO_TRAIN_IMAGE_DIR, YOLO_VAL_IMAGE_DIR,
             YOLO_TEST_LABEL_DIR, YOLO_TRAIN_LABEL_DIR, YOLO_VAL_LABEL_DIR]
for directory in yolo_dirs:
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
# Start dataset creation
create_labels(COCO_TEST_DIR, YOLO_TEST_LABEL_DIR)
create_labels(COCO_TRAIN_DIR, YOLO_TRAIN_LABEL_DIR)
create_labels(COCO_VAL_DIR, YOLO_VAL_LABEL_DIR)

copy_images(COCO_TEST_DIR, YOLO_TEST_IMAGE_DIR)
copy_images(COCO_TRAIN_DIR, YOLO_TRAIN_IMAGE_DIR)
copy_images(COCO_VAL_DIR, YOLO_VAL_IMAGE_DIR)